# Lesson 24: Human-in-the-Loop for Brown Writing Workflow

In this lesson, we'll explore how to implement human-in-the-loop capabilities in the Brown writing workflow. We'll learn how to integrate human feedback into the article review and editing process, expose the workflows as MCP tools for seamless integration with AI assistants like Claude and Cursor, and create a collaborative writing experience where AI and human expertise combine.

**Learning Objectives:**

- Understand the importance of human feedback and human-in-the-loop in AI writing workflows
- Learn how to implement the HumanFeedback entity and integrate it into the ArticleReviewer node
- Explore two new editing workflows: edit article and edit selected text
- Discover how to expose Brown as an MCP server with tools, prompts, and resources
- See how to integrate Brown with MCP clients like Cursor for a coding-like writing experience

> [!NOTE]
> 💡 Remember that you can also run `brown` as a standalone Python package by going to `lessons/writing_workflow/` and following the instructions from there.

## 1. Setup

First, we define some standard Magic Python commands to autoreload Python packages whenever they change:

In [1]:
%load_ext autoreload
%autoreload 2

### Set Up Python Environment

To set up your Python virtual environment using `uv` and load it into the Notebook, follow the step-by-step instructions from the `Course Admin` lesson from the beginning of the course.

**TL/DR:** Be sure the correct kernel pointing to your `uv` virtual environment is selected.

### Configure Gemini API

To configure the Gemini API, follow the step-by-step instructions in the `Course Admin` lesson.

Here is a quick checklist of what you need to run this notebook:

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/api-keys).
2.  From the root of your project, run: `cp .env.example .env` 
3.  Within the `.env` file, fill in the `GOOGLE_API_KEY` variable:

Now, the code below will load the key from the `.env` file:

In [2]:
from utils import env

env.load(required_env_vars=["GOOGLE_API_KEY"])

Environment variables loaded from `/Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.env`
Environment variables loaded successfully.


### Import Key Packages

In [3]:
import nest_asyncio
from utils import pretty_print

nest_asyncio.apply()  # Allow nested async usage in notebooks

### Download Required Files

We need to download the configuration files and input data that Brown uses for article generation and editing.

First, let's download the configs folder:

In [4]:
%%capture

!rm -rf configs
!curl -L -o configs.zip https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/data/configs.zip
!unzip configs.zip
!rm -rf configs.zip

Now, let's download the inputs folder containing profiles, examples, and test data:

In [5]:
%%capture

!rm -rf inputs
!curl -L -o inputs.zip https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/data/inputs.zip
!unzip inputs.zip
!rm -rf inputs.zip

Let's verify what we downloaded:

In [6]:
%ls

article_guideline.md   inputs/                notebook_guideline.md
configs/               notebook.ipynb


Now let's define constants to reference these directories throughout the notebook:

In [7]:
from pathlib import Path

CONFIGS_DIR = Path("configs")
INPUTS_DIR = Path("inputs")

print(f"Configs directory exists: {CONFIGS_DIR.exists()}")
print(f"Inputs directory exists: {INPUTS_DIR.exists()}")

Configs directory exists: True
Inputs directory exists: True


In [8]:
EXAMPLES_DIR = Path("inputs/examples/course_lessons")
PROFILES_DIR = Path("inputs/profiles")

print(f"Examples directory exists: {EXAMPLES_DIR.exists()}")
print(f"Profiles directory exists: {PROFILES_DIR.exists()}")

Examples directory exists: True
Profiles directory exists: True


First, we will load a simpler example that runs faster and is easier to understand. At the end, we will load a larger sample that is closer to what we do on our end to generate professional articles:

In [9]:
SAMPLE_DIR = Path("inputs/tests/01_sample_small")

print(f"Samples directory exists: {SAMPLE_DIR.exists()}")

Samples directory exists: True


## 2. Adding Human-In-The-Loop In Our Writing Workflow

After generating an article using the writing workflow we explained in Lessons 22 and 23, you'll likely want to refine it further. Writing is highly subjective, and even the best AI-generated content benefits from human review and editing.

The perfect balance between AI and human expertise is to use AI to generate and automate parts of your work, then have you, as the domain expert, review and refine it. Known as the AI generation - human validation loop. 

This is exactly what we've designed the Brown writing workflow to support.

### The Human-in-the-Loop Design

We designed Brown to easily introduce humans into the loop between generating the first version of an article and refining it through additional review and editing cycles with human feedback. This means:

1. We can use a low number of review loops during initial article generation to reduce costs and latency
2. After reviewing the generated article, we can dynamically run additional review and editing workflows with human feedback
3. We can edit either the entire article or just selected sections based on your needs

### Decoupling Workflows with MCP

To enable this human-in-the-loop approach, we needed to decouple the article generation workflow from the editing workflows. We used MCP servers to achieve this separation, where:

- The `generate_article` workflow is one independent MCP tool
- The `edit_article` workflow is another independent MCP tool
- The `edit_selected_text` workflow is a third independent MCP tool

This architecture allows you to generate an article, review it, and then selectively apply additional editing workflows with your human feedback until you're satisfied with the results.

Here's how the workflow looks:

<img src="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/l24_writing_workflow.png" alt="Workflow" height="700"/>

The diagram shows how Brown as an MCP server exposes three main tools, with a human feedback loop that allows iterative refinement until you're satisfied with the article.

## 3. Introducing Human Feedback Into the Article Reviewer

Let's see how we introduced human feedback into our Article Reviewer Node. We'll start by explaining the `HumanFeedback` entity, then show how it's integrated into the `ArticleReviewer` node, and finally demonstrate it with a working example.

### 3.1 The HumanFeedback Entity

The `HumanFeedback` entity is a simple but Pydantic model that encapsulates human feedback for the article review process.

Source: `brown.entities.reviews`
```python
class HumanFeedback(BaseModel, ContextMixin):
    content: str

    def to_context(self) -> str:
        return f"""
<{self.xml_tag}>
    {self.content}
</{self.xml_tag}>
"""
```

### 3.2 Human Feedback in ArticleReviewer

Now let's see how the `ArticleReviewer` node integrates human feedback into the review process. We'll focus only on the relevant sections.

Source: `brown.nodes.article_reviewer`

1. **Initialization with Human Feedback**
```python
def __init__(
    self,
    to_review: Article | SelectedText,
    article_guideline: ArticleGuideline,
    model: Runnable,
    article_profiles: ArticleProfiles,
    human_feedback: HumanFeedback | None = None,
) -> None:
    self.to_review = to_review
    self.article_guideline = article_guideline
    self.article_profiles = article_profiles
    self.human_feedback = human_feedback

    super().__init__(model, toolkit=Toolkit(tools=[]))
```

The `ArticleReviewer` now accepts an optional `human_feedback` parameter. This allows the reviewer to work with or without human input.

2. **Human Feedback in the System Prompt**

The system prompt includes a dedicated section for human feedback:
```python
system_prompt_template = """
You are Brown, an expert article writer, editor and reviewer specialized in reviewing technical, educative and informational articles.

...

## Human Feedback

Along with the expected requirements, a human already reviewed the article and provided the following feedback:

{human_feedback}

If empty, completely ignore it, otherwise the feedback will ALWAYS be used in two ways:
1. First you will use the <human_feedback> to guide your reviewing process against the requirements. This will help you understand 
on what rules to focus on as this directly highlights what the user wants to improve.
2. Secondly you will extract one or more action points based on the <human_feedback>. Depending on how many ideas, topics or suggestions 
the <human_feedback> contains you will generate from 1 to N action points. Each <human_feedback> review will contain a single action point. 
3. As long the <human_feedback> is not empty, you will always return at least 1 action point, but you will return more action points 
if the feedback touches multiple ideas. 

Here is an example of a reviewed based on the human feedback:
<example_of_human_feedback_action_point>
Review(
    profile="human_feedback",
    location="Article level",
    comment="Add all the points from the article guideline to the article."
)
</example_of_human_feedback_action_point>

...
"""
```

This section instructs the LLM on how to use human feedback:
- Use it to guide the review process and focus on specific rules
- Extract action points from the feedback (1 to N depending on how many ideas are present)
- Always return at least 1 action point if feedback is provided
- Each action point becomes a review with `profile="human_feedback"`

3. **Injecting Human Feedback into the Prompt**

When the reviewer runs, it injects the human feedback into the system prompt:

```python
async def ainvoke(self) -> ArticleReviews | SelectedTextReviews:
    system_prompt = self.system_prompt_template.format(
        human_feedback=self.human_feedback.to_context() if self.human_feedback else "",
        article=self.article.to_context(),
        article_guideline=self.article_guideline.to_context(),
        character_template=self.article_profiles.character.to_context(),
        article_template=self.article_profiles.article.to_context(),
        structure_template=self.article_profiles.structure.to_context(),
        mechanics_template=self.article_profiles.mechanics.to_context(),
        terminology_template=self.article_profiles.terminology.to_context(),
        tonality_template=self.article_profiles.tonality.to_context(),
    )
    ...
```

If `human_feedback` is provided, it's converted to XML context format and injected. Otherwise, an empty string is used.

### 3.3 Example: Using ArticleReviewer with Human Feedback

Let's see a practical example of using the `ArticleReviewer` with human feedback. We'll load our sample article, article guideline, and profiles, then provide human feedback to guide the review process.

First, let's import the necessary components:

In [10]:
from brown.entities.reviews import HumanFeedback
from brown.loaders import (
    MarkdownArticleExampleLoader,
    MarkdownArticleGuidelineLoader,
    MarkdownArticleLoader,
    MarkdownArticleProfilesLoader,
)
from brown.models import SupportedModels, get_model
from brown.nodes.article_reviewer import ArticleReviewer

2025-11-26 17:26:07.324 | INFO     | brown.config:<module>:10 - Loading environment file from `.env`


Now let's load the sample inputs. We'll use the same article and guidelines from the test sample directory as we used in previous lessons:

In [11]:
pretty_print.wrapped("STEP 1: Loading Context", width=100)

# Load guideline
guideline_loader = MarkdownArticleGuidelineLoader(uri=Path("article_guideline.md"))
article_guideline = guideline_loader.load(working_uri=SAMPLE_DIR)

# Load profiles
profiles_input = {
    "article": PROFILES_DIR / "article_profile.md",
    "character": PROFILES_DIR / "character_profiles" / "paul_iusztin.md",
    "mechanics": PROFILES_DIR / "mechanics_profile.md",
    "structure": PROFILES_DIR / "structure_profile.md",
    "terminology": PROFILES_DIR / "terminology_profile.md",
    "tonality": PROFILES_DIR / "tonality_profile.md",
}
profiles_loader = MarkdownArticleProfilesLoader(uri=profiles_input)
profiles = profiles_loader.load()

# Load examples
examples_loader = MarkdownArticleExampleLoader(uri=EXAMPLES_DIR)
article_examples = examples_loader.load()

article_loader = MarkdownArticleLoader(uri="article.md")
article = article_loader.load(working_uri=SAMPLE_DIR)

print(f"✓ Guideline: {len(article_guideline.content):,} characters")
print(f"✓ Article: {len(article.content):,} characters")
print(f"✓ Profiles: {len(profiles_input)} profiles loaded")
print(f"✓ Examples: {len(article_examples.examples)} article examples")

----------------------------------------------------------------------------------------------------
  STEP 1: Loading Context
----------------------------------------------------------------------------------------------------
✓ Guideline: 6,751 characters
✓ Article: 6,982 characters
✓ Profiles: 6 profiles loaded
✓ Examples: 2 article examples


Here is a reminder on how the article looks like:

In [12]:
pretty_print.wrapped(f"{article.to_context()[:4000]}", title="Article (first 4000 characters)")

--------------------------------- Article (first 4000 characters) ---------------------------------
  
<article>
    # Workflows vs. Agents: The Critical Decision Every AI Engineer Faces
### How to choose between predictable control and autonomous flexibility when building AI applications.

When building AI applications, engineers face a critical architectural decision early on. Should you create a predictable, step-by-step workflow where you control every action, or build an autonomous agent that can think and decide for itself? This choice impacts everything from development time and cost to reliability and user experience. It is a fundamental decision that often determines if an AI application will be successful in production.

By the end of this lesson, you will understand the fundamental differences between LLM workflows and AI agents, know when to use each, and recognize how to combine their strengths in hybrid approaches.

## Understanding the Spectrum: From Workflows to Agents


Now let's create human feedback and run the article reviewer:

In [13]:
human_feedback = HumanFeedback(
    content="""Make the introduction more engaging and catchy. 
Also, expand on the definition of both workflows and agents from the first section"""
)

# Create the article reviewer
model = get_model(SupportedModels.GOOGLE_GEMINI_25_FLASH)
article_reviewer = ArticleReviewer(
    to_review=article,
    article_guideline=article_guideline,
    model=model,
    article_profiles=profiles,
    human_feedback=human_feedback,
)

print("Running article review with human feedback...")
reviews = await article_reviewer.ainvoke()
print(f"\nGenerated {len(reviews.reviews)} reviews")

Running article review with human feedback...

Generated 19 reviews


Let's examine the reviews, especially focusing on the human feedback reviews:

In [14]:
from utils import pretty_print

# Print human feedback reviews
human_feedback_reviews = [r for r in reviews.reviews if r.profile == "human_feedback"]
pretty_print.wrapped(
    f"Found {len(human_feedback_reviews)} reviews based on human feedback", title="Human Feedback Reviews"
)

for i, review in enumerate(human_feedback_reviews, 1):
    pretty_print.wrapped(
        {"profile": review.profile, "location": review.location, "comment": review.comment},
        title=f"{i}. Human Feedback Review",
    )

-------------------------------------- Human Feedback Reviews --------------------------------------
  Found 2 reviews based on human feedback
----------------------------------------------------------------------------------------------------
------------------------------------- 1. Human Feedback Review -------------------------------------
  {
  "profile": "human_feedback",
  "location": "Article level",
  "comment": "Make the introduction more engaging and catchy."
}
----------------------------------------------------------------------------------------------------
------------------------------------- 2. Human Feedback Review -------------------------------------
  {
  "profile": "human_feedback",
  "location": "Understanding the Spectrum: From Workflows to Agents - First section",
  "comment": "Expand on the definition of both workflows and agents from the first section."
}
----------------------------------------------------------------------------------------------------


Let's also see all the other reviews from the profiles:

In [15]:
profile_types = set(r.profile for r in reviews.reviews)

pretty_print.wrapped(
    f"Generated reviews from {len(profile_types)} different profile types: {', '.join(sorted(profile_types))}",
    title="All Reviews Summary",
)
print()

for profile_type in sorted(profile_types):
    profile_reviews = [r for r in reviews.reviews if r.profile == profile_type]
    pretty_print.wrapped(f"{profile_type.upper()}: {len(profile_reviews)} reviews")
    for i, review in enumerate(profile_reviews[:2], 1):  # Show first 2 of each type
        print(f"  {i}. [{review.location}] {review.comment[:100]}...")
    print()

--------------------------------------- All Reviews Summary ---------------------------------------
  Generated reviews from 7 different profile types: article_guideline, article_profile, human_feedback, mechanics_profile, structure_profile, terminology_profile, tonality_profile
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
  ARTICLE_GUIDELINE: 7 reviews
----------------------------------------------------------------------------------------------------
  1. [Introduction: The Critical Decision Every AI Engineer Faces - Article level] The introduction's length is 123 words, exceeding the guideline of 100 words....
  2. [Understanding the Spectrum: From Workflows to Agents - Second paragraph] The article states 'We will look at their core properties and how they are used, rather than their t...

-------------------------------------

Notice how the reviewer generated reviews from multiple sources:
- Human feedback reviews that directly address your specific requests
- Profile-based reviews (article, structure, mechanics, terminology, tonality) that ensure adherence to the style guidelines

The human feedback reviews always have `profile="human_feedback"` and create action points based on your feedback. These reviews will be used by the article writer to edit the article according to your instructions.

## 4. Implementing the Article Editing Workflow

Now that we understand how human feedback integrates with the article reviewer, let's explore the `edit_article` workflow. This workflow reviews and edits an existing article based on human feedback and the expected requirements.

The edit article workflow contains only one loop of the same reviewing-editing logic we already use within the generate article workflow. 

Also, the edit article workflow follows the same clean architecture pattern we've used throughout Brown. It leverages the app layer to orchestrate nodes and entities, keeping the code modular and maintainable.

### 4.1 Building the Edit Article Workflow

The workflow is built using LangGraph's functional API. Here's how it's structured:

Source: `brown.workflows.edit_article`
```python
def build_edit_article_workflow(checkpointer: BaseCheckpointSaver):
    """Create an edit article workflow with checkpointer.

    Args:
        checkpointer: Checkpointer to use for workflow persistence.

    Returns:
        Configured workflow entrypoint
    """

    return entrypoint(checkpointer=checkpointer)(_edit_article_workflow)
```

The `build_edit_article_workflow` function is a factory that creates the workflow with a checkpointer for persistence. It uses LangGraph's `@entrypoint` decorator to wrap the main workflow function.

The workflow expects an `EditArticleInput` typed dictionary:

```python
class EditArticleInput(TypedDict):
    dir_path: Path
    human_feedback: str
```

This input specifies:
- `dir_path`: The directory containing the article and all supporting files (guideline, profiles, research, etc.)
- `human_feedback`: The human feedback string to guide the editing process

### 4.2 The Edit Article Workflow Logic

The main workflow function orchestrates the entire editing process:
```python
async def _edit_article_workflow(inputs: EditArticleInput, config: RunnableConfig) -> str:
    writer = get_stream_writer()

    # Progress: Loading context
    writer(WorkflowProgress(progress=0, message="Loading context").model_dump(mode="json"))
    context = {}
    loaders = build_loaders(app_config)
    for context_name, loader in loaders.items():
        loader = cast(Loader, loader)
        context[context_name] = loader.load(working_uri=inputs["dir_path"])

    human_feedback = HumanFeedback(content=inputs["human_feedback"])
    writer(WorkflowProgress(progress=5, message="Loaded context").model_dump(mode="json"))

    # Progress: Reviewing
    writer(WorkflowProgress(progress=20, message="Reviewing article").model_dump(mode="json"))
    reviews = await generate_reviews(context["article"], human_feedback, context["article_guideline"], context["profiles"])
    writer(WorkflowProgress(progress=40, message="Generated reviews").model_dump(mode="json"))

    # Progress: Editing
    writer(WorkflowProgress(progress=60, message="Editing article").model_dump(mode="json"))
    article = await edit_based_on_reviews(
        context["article_guideline"], context["research"], context["profiles"], context["examples"], reviews
    )
    writer(WorkflowProgress(progress=80, message="Edited article").model_dump(mode="json"))

    # Progress: Complete
    writer(WorkflowProgress(progress=100, message="Article editing completed").model_dump(mode="json"))

    return f"""
Here is the edited article:
{article.to_context()}

Here is what you have to do with the edited article:
- print the edited article to the console for the user to see
- give a quick summary of the changes you made
- always apply the changes to the source file without waiting for the user to explicitly ask you to do so
- in case you haven't changed anything, just say that you haven't changed anything
"""
```

The workflow follows these steps:
1. **Load context**: Use the loader builders to load the article, guideline, profiles, research, and examples from the directory
2. **Create human feedback**: Convert the feedback string into a `HumanFeedback` entity
3. **Generate reviews**: Run the article reviewer with human feedback to generate reviews
4. **Edit based on reviews**: Run the article writer with the reviews to produce an edited article
5. **Return instructions**: Return the edited article along with instructions for the MCP client on what to do next

Notice how steps 3 and 4 are identical to the ones from the writing workflow you learned in lesson 23.

### 4.3 Generating Reviews

The `generate_reviews` task creates reviews by running the `ArticleReviewer` node:
```python
@task(retry_policy=retry_policy)
async def generate_reviews(
    article: Article,
    human_feedback: HumanFeedback,
    article_guideline: ArticleGuideline,
    article_profiles: ArticleProfiles,
) -> ArticleReviews:
    model, _ = build_model(app_config, node="review_article")
    article_reviewer = ArticleReviewer(
        to_review=article,
        article_guideline=article_guideline,
        article_profiles=article_profiles,
        human_feedback=human_feedback,
        model=model,
    )
    reviews = await article_reviewer.ainvoke()

    return cast(ArticleReviews, reviews)
```

This task:
- Builds the model from the app config for the "review_article" node
- Creates an `ArticleReviewer` with the article, guideline, profiles, and human feedback
- Uses LangGraph's `@task` decorator with a retry policy for resilience

### 4.4 Editing Based on Reviews

The `edit_based_on_reviews` task creates an edited article using the `ArticleWriter` node:
```python
@task(retry_policy=retry_policy)
async def edit_based_on_reviews(
    article_guideline: ArticleGuideline,
    research: Research,
    article_profiles: ArticleProfiles,
    article_examples: ArticleExamples,
    reviews: ArticleReviews,
) -> Article:
    model, _ = build_model(app_config, node="edit_article")
    article_writer = ArticleWriter(
        article_guideline=article_guideline,
        research=research,
        article_profiles=article_profiles,
        media_items=MediaItems.build(),
        article_examples=article_examples,
        reviews=reviews,
        model=model,
    )
    article = await article_writer.ainvoke()

    return cast(Article, article)
```

This task:
- Builds the model from the app config for the "edit_article" node
- Creates an `ArticleWriter` with all necessary context and the reviews to address
- Also uses the `@task` decorator with retry policy

Notice how the `ArticleWriter` works in "editing mode" when provided with `reviews`. It uses the same writer node from article generation, but the reviews guide it to make specific changes rather than writing from scratch.

### 4.5 Running the Edit Article Workflow

In [16]:
import uuid

from brown.memory import build_in_memory_checkpointer
from brown.workflows.edit_article import build_edit_article_workflow

async with build_in_memory_checkpointer() as checkpointer:
    print("1. Building workflow...\n")
    workflow = build_edit_article_workflow(checkpointer=checkpointer)

    print("2. Configuring workflow...\n")
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}
    print(f"   ✓ Thread ID: {thread_id}")

    print("3. Running workflow...")
    print("   This will take several minutes...\n")

    async for event in workflow.astream(
        {
            "dir_path": SAMPLE_DIR,
            "human_feedback": """
Make the introduction more engaging, catchy and shorter. 
Also, expand on the definition of both workflows and agents from the first section""",
        },
        config=config,
        stream_mode=["custom", "values"],
    ):
        event_type, event_data = event
        if event_type == "custom":
            pretty_print.wrapped(event_data, title="Event")
        elif event_type == "values":
            pretty_print.wrapped(event_data, title="Output")

pretty_print.wrapped("WORKFLOW COMPLETED", width=100)

1. Building workflow...

2. Configuring workflow...

   ✓ Thread ID: a8309001-6f52-48a0-b2b7-92f73d5f005e
3. Running workflow...
   This will take several minutes...

---------------------------------------------- Event ----------------------------------------------
  {
  "progress": 0,
  "message": "Loading context"
}
----------------------------------------------------------------------------------------------------
---------------------------------------------- Event ----------------------------------------------
  {
  "progress": 5,
  "message": "Loaded context"
}
----------------------------------------------------------------------------------------------------
---------------------------------------------- Event ----------------------------------------------
  {
  "progress": 20,
  "message": "Reviewing article"
}
----------------------------------------------------------------------------------------------------
---------------------------------------------- Event -------------

As you can see, the workflow's output is the edited article. 

We will explain in more depth the impact of running the review-editing workflow alongside human feedback in the video at the end of the lesson.

### 4.6 The Power of Human Feedback

The edit article workflow demonstrates a key advantage of our architecture:

**We can use a low number of review loops during initial article generation, and further run them dynamically with a human in the loop when necessary, with more human guidance.**

This means:
- Initial generation is faster and cheaper with fewer automatic review iterations
- We don't assume how many iterations we need to have an ideal output, but let you decide
- The workflow runs additional review and editing cycles guided by your feedback
- You can repeat this process until satisfied with the results

This approach balances efficiency with quality, using AI to handle the heavy lifting while keeping you in control of the final output.

## 5. Implementing the Selected Text Editing Workflow

While the edit article workflow handles entire article edits, you'll often want to refine just a specific section. The `edit_selected_text` workflow enables precise, focused edits on selected text portions.

The workflow structure is almost identical to `edit_article`, thanks to our clean architecture. The main difference is that it operates on a `SelectedText` entity instead of the full `Article`.

### 5.1 Building the Edit Selected Text Workflow

The workflow builder follows the same pattern:

Source: `brown.workflows.edit_selected_text`
```python
def build_edit_selected_text_workflow(checkpointer: BaseCheckpointSaver):
    """Create an edit selected text workflow with checkpointer.

    Args:
        checkpointer: Checkpointer to use for workflow persistence.

    Returns:
        Configured workflow entrypoint
    """

    return entrypoint(checkpointer=checkpointer)(_edit_selected_text_workflow)
```

The workflow expects an `EditSelectedTextInput` typed dictionary:

```python
class EditSelectedTextInput(TypedDict):
    dir_path: Path
    human_feedback: str
    selected_text: str
    number_line_before_selected_text: int
    number_line_after_selected_text: int
```

This input specifies:
- `dir_path`: The directory containing the article and supporting files
- `human_feedback`: Human feedback to guide the editing
- `selected_text`: The specific text portion to edit
- `number_line_before_selected_text`: The starting line number in the article
- `number_line_after_selected_text`: The ending line number in the article

The line numbers help the workflow locate the selected text within the larger article context.

### 5.2 The Edit Selected Text Workflow Logic

The main workflow function is structurally similar to `edit_article`:
```python
async def _edit_selected_text_workflow(inputs: EditSelectedTextInput, config: RunnableConfig) -> str:
    writer = get_stream_writer()

    # Progress: Loading context
    writer(WorkflowProgress(progress=0, message="Loading context").model_dump(mode="json"))
    context = {}
    loaders = build_loaders(app_config)
    for context_name, loader in loaders.items():
        loader = cast(Loader, loader)
        context[context_name] = loader.load(working_uri=inputs["dir_path"])

    selected_text = SelectedText(
        article=context["article"],
        content=inputs["selected_text"],
        first_line_number=inputs["number_line_before_selected_text"],
        last_line_number=inputs["number_line_after_selected_text"],
    )
    human_feedback = HumanFeedback(content=inputs["human_feedback"])
    writer(WorkflowProgress(progress=5, message="Loaded context").model_dump(mode="json"))

    # Progress: Reviewing
    writer(WorkflowProgress(progress=20, message="Reviewing selected text").model_dump(mode="json"))
    reviews = await generate_reviews(selected_text, human_feedback, context["article_guideline"], context["profiles"])
    writer(WorkflowProgress(progress=40, message="Generated reviews").model_dump(mode="json"))

    # Progress: Editing
    writer(WorkflowProgress(progress=60, message="Editing selected text").model_dump(mode="json"))
    selected_text = await edit_based_on_reviews(
        context["article_guideline"], context["research"], context["profiles"], context["examples"], reviews
    )
    writer(WorkflowProgress(progress=80, message="Edited selected text").model_dump(mode="json"))

    # Progress: Complete
    writer(WorkflowProgress(progress=100, message="Selected text editing completed").model_dump(mode="json"))

    return f"""
Here is the edited selected text:
{selected_text.to_context()}

Here is what you have to do with edited selected text:
- print the edited selected text to the console for the user to see
- give a quick summary of the changes you made
- always apply the changes to the source file without waiting for the user to explicitly ask you to do so
- in case you haven't changed anything, just say that you haven't changed anything
"""
```

The workflow follows these steps:
1. **Load context**: Load the full article and supporting files
2. **Create selected text entity**: Build a `SelectedText` entity that contains the selected portion, the full article for context, and line numbers
3. **Create human feedback**: Convert the feedback string to a `HumanFeedback` entity
4. **Generate reviews**: Review the selected text with human feedback
5. **Edit based on reviews**: Edit the selected text based on the reviews
6. **Return instructions**: Return the edited selected text with instructions

### 5.3 Generating Reviews for Selected Text

The `generate_reviews` task for selected text is nearly identical to the article version:
```python
@task(retry_policy=retry_policy)
async def generate_reviews(
    selected_text: SelectedText,
    human_feedback: HumanFeedback,
    article_guideline: ArticleGuideline,
    article_profiles: ArticleProfiles,
) -> SelectedTextReviews:
    model, _ = build_model(app_config, node="review_selected_text")
    selected_text_reviewer = ArticleReviewer(
        to_review=selected_text,
        human_feedback=human_feedback,
        article_guideline=article_guideline,
        article_profiles=article_profiles,
        model=model,
    )
    reviews = await selected_text_reviewer.ainvoke()

    return cast(SelectedTextReviews, reviews)
```

The key difference is:
- It takes a `SelectedText` instead of `Article`
- It returns `SelectedTextReviews` instead of `ArticleReviews`
- It uses the "review_selected_text" node config

The `ArticleReviewer` node is smart enough to handle both cases. When given a `SelectedText`, it focuses reviews on that portion while using the full article as context.

### 5.4 Editing Selected Text Based on Reviews

The `edit_based_on_reviews` task for selected text also follows the same pattern:
```python
@task(retry_policy=retry_policy)
async def edit_based_on_reviews(
    article_guideline: ArticleGuideline,
    research: Research,
    article_profiles: ArticleProfiles,
    article_examples: ArticleExamples,
    reviews: SelectedTextReviews,
) -> SelectedText:
    model, _ = build_model(app_config, node="edit_selected_text")
    article_writer = ArticleWriter(
        article_guideline=article_guideline,
        research=research,
        article_profiles=article_profiles,
        media_items=MediaItems.build(),
        article_examples=article_examples,
        reviews=reviews,
        model=model,
    )
    edited_selected_text = cast(SelectedText, await article_writer.ainvoke())

    return edited_selected_text
```

This task:
- Takes `SelectedTextReviews` instead of `ArticleReviews`
- Returns `SelectedText` instead of `Article`
- Uses the "edit_selected_text" node config

Again, the `ArticleWriter` node handles both article and selected text editing seamlessly.

### 5.5 Why Edit Selected Text?

The edit selected text workflow is crucial because:

**Most often we don't want to edit the whole article, but just a small section, or apply the human feedback just to a small section.**

This workflow enables:
- Faster and cheaper edits by focusing on specific sections
- More precise changes without affecting other parts of the article
- Iterative refinement of individual paragraphs or sections
- Better control over the editing process

Combined with the edit article workflow, you have complete flexibility to refine content at any granularity you need.

### 5.6 Running the Edit Selected Text Workflow

First, explictly load the selected text that we want to edit from the sample article:

In [17]:
article = MarkdownArticleLoader(uri="article.md").load(working_uri=SAMPLE_DIR)

start_line = 8
end_line = 42
selected_text = "\n".join(article.content.split("\n")[start_line:end_line])
pretty_print.wrapped(selected_text, title="Selected text to edit")

-------------------------------------- Selected text to edit --------------------------------------
  
To make the right choice, you first need to understand what LLM workflows and AI agents are. We will look at their core properties and how they are used, rather than their technical specifics.

### LLM Workflows

An LLM workflow is a sequence of tasks orchestrated by developer-written code. It can include LLM calls, but also other operations like reading from a database or calling an API. Think of it like a recipe where each step is explicitly defined. The key characteristic is that the path is determined in advance, resulting in a deterministic or rule-based system. This gives you predictable execution, explicit control over the application's flow, and makes the system easier to test and debug. Because you control every step, you know exactly where a failure occurred and how to fix it.

```mermaid
graph TD
    A["Start"] --> B["LLM Call"]
    B --> C["Process Data"]
    C --> D["Stor

Then, call the workflow:

In [18]:
from brown.workflows.edit_selected_text import build_edit_selected_text_workflow

async with build_in_memory_checkpointer() as checkpointer:
    print("1. Building workflow...\n")
    workflow = build_edit_selected_text_workflow(checkpointer=checkpointer)

    print("2. Configuring workflow...\n")
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}
    print(f"   ✓ Thread ID: {thread_id}")

    print("3. Running workflow...")
    print("   This will take several minutes...\n")
    async for event in workflow.astream(
        {
            "dir_path": SAMPLE_DIR,
            "human_feedback": "Expand on the definition of both workflows and agents.",
            "selected_text": selected_text,
            "number_line_before_selected_text": start_line,
            "number_line_after_selected_text": end_line,
        },
        config=config,
        stream_mode=["custom", "values"],
    ):
        event_type, event_data = event
        if event_type == "custom":
            pretty_print.wrapped(event_data, title="Event")
        elif event_type == "values":
            pretty_print.wrapped(event_data, title="Output")

pretty_print.wrapped("WORKFLOW COMPLETED", width=100)

1. Building workflow...

2. Configuring workflow...

   ✓ Thread ID: f281995f-fef6-4e9f-8887-9cedc09c0710
3. Running workflow...
   This will take several minutes...

---------------------------------------------- Event ----------------------------------------------
  {
  "progress": 0,
  "message": "Loading context"
}
----------------------------------------------------------------------------------------------------
---------------------------------------------- Event ----------------------------------------------
  {
  "progress": 5,
  "message": "Loaded context"
}
----------------------------------------------------------------------------------------------------
---------------------------------------------- Event ----------------------------------------------
  {
  "progress": 20,
  "message": "Reviewing selected text"
}
----------------------------------------------------------------------------------------------------
---------------------------------------------- Event -------

As before, the workflow's output is the edited piece of selected text. As with the edit article workflow, we will explain in more depth the impact of running the review-editing workflow alongside human feedback in the video at the end of the lesson.

Now, let's move one step closer and see how to properly integrate these two new workflows into a person's daily workflow by serving them as an MCP Server. 

## 6. Serving Brown as an MCP Server

Now we'll see how Brown exposes all its workflows as an MCP server. This allows external applications like Claude Desktop and Cursor to use Brown's capabilities through the Model Context Protocol.

All the MCP code lives in the `brown.mcp` module, keeping the serving layer completely separate from the domain, app, and infrastructure layers. This separation allows us to potentially serve Brown through different interfaces (CLI, FastAPI, etc.) without changing the core logic.

### 6.1 MCP Server and Tools

The MCP server is built using FastMCP (the same as the Nova Deep Research Agent, the first capstone project) and exposes three main tools:

Source: `brown.mcp.server`
```python
from fastmcp import FastMCP

mcp = FastMCP("Brown MCP Server")

@mcp.tool
async def generate_article(dir_path: Path, ctx: Context) -> str:
    """Generate an article from scratch using Brown's article generation workflow.
    
    Args:
        dir_path: Path to the directory containing article resources
        ctx: MCP context for streaming progress updates
        
    Returns:
        A string containing a success confirmation message
    """
    async with build_short_term_memory(app_config) as checkpointer:
        generate_article_workflow = build_generate_article_workflow(checkpointer=checkpointer)
        
        thread_id = str(uuid.uuid4())
        config = {"configurable": {"thread_id": thread_id}}
        
        async for chunk in generate_article_workflow.astream({"dir_path": dir_path}, config=config, stream_mode=["custom", "values"]):
            _, chunk_data = chunk
            await parse_message(chunk_data, ctx)
    
    return "Article generation completed successfully!"

@mcp.tool
async def edit_article(
    article_path: str,
    human_feedback: str,
    ctx: Context,
) -> str:
    """Edit an entire article based on human feedback.
    
    Args:
        article_path: Path to the article that has to be edited
        human_feedback: User's feedback or instructions for editing
        ctx: MCP context for streaming progress updates
        
    Returns:
        The fully edited article content plus instructions
    """
    async with build_short_term_memory(app_config) as checkpointer:
        edit_article_workflow = build_edit_article_workflow(checkpointer=checkpointer)
        
        dir_path = Path(article_path).parent
        thread_id = str(uuid.uuid4())
        config = {"configurable": {"thread_id": thread_id}}
        
        final_result = None
        async for chunk in edit_article_workflow.astream(
            {"dir_path": Path(dir_path), "human_feedback": human_feedback},
            config=config,
            stream_mode=["custom", "values"],
        ):
            chunk_type, chunk_data = chunk
            await parse_message(chunk_data, ctx)
            
            if chunk_type == "values":
                final_result = chunk_data
        
    return final_result or "Article editing completed successfully!"

@mcp.tool
async def edit_selected_text(
    article_path: str,
    human_feedback: str,
    selected_text: str,
    first_line_number: int,
    last_line_number: int,
    ctx: Context,
) -> str:
    """Edit a selected section of an article based on human feedback.
    
    Args:
        article_path: Path to the article containing the selected text
        human_feedback: User's feedback or instructions
        selected_text: The specific text selected from the article
        first_line_number: Line number where the selected text starts
        last_line_number: Line number where the selected text ends
        ctx: MCP context for streaming progress updates
        
    Returns:
        The fully edited selected text plus instructions
    """
    async with build_short_term_memory(app_config) as checkpointer:
        edit_selected_text_workflow = build_edit_selected_text_workflow(checkpointer=checkpointer)
        
        dir_path = Path(article_path).parent
        thread_id = str(uuid.uuid4())
        config = {"configurable": {"thread_id": thread_id}}
        
        final_result = None
        async for chunk in edit_selected_text_workflow.astream(
            {
                "dir_path": Path(dir_path),
                "human_feedback": human_feedback,
                "selected_text": selected_text,
                "number_line_before_selected_text": first_line_number,
                "number_line_after_selected_text": last_line_number,
            },
            config=config,
            stream_mode=["custom", "values"],
        ):
            chunk_type, chunk_data = chunk
            await parse_message(chunk_data, ctx)
            
            if chunk_type == "values":
                final_result = chunk_data
        
    return final_result or "Selected text editing completed successfully!"
```

**Each tool:**

- Builds the appropriate workflow with an in memory checkpointer, similar to how we ran the workflows so far in these lessons
- Creates a unique thread ID for each workflow execution
- Most importantly, has detailed pydocs and signatures that will be used by the MCP client to understand what tool to call and how to call it. 

- Streams progress updates to the MCP client via the `ctx` (context) parameter

Before running one of these tools, let's take a look at how we report the progress from each tool through `parse_message` function:
```python
async def parse_message(chunk_data: dict, ctx: Context, prefix: str = "") -> None:
    """Parse and report workflow streaming messages to the MCP client.

    Args:
        chunk_data: The streaming data from the workflow, can be a string message
                   or a dictionary containing progress information.
        ctx: MCP context for sending messages and progress updates to the client.
        prefix: Optional prefix to add to all messages for context identification.

    Raises:
        ValueError: If chunk_data is not a supported type (str or dict).
    """

    if prefix:
        prefix = f"{prefix}: "

    if isinstance(chunk_data, str):
        await ctx.info(f"{prefix}{chunk_data}")
    elif isinstance(chunk_data, dict):
        message = WorkflowProgress(**chunk_data)
        await ctx.info(f"{prefix}{message.progress}%: {message.message}")
        await ctx.report_progress(progress=message.progress, total=100, message=f"{prefix}{message.message}")
    else:
        raise ValueError(f"Unsupported chunk data type: {type(chunk_data)}")
```

Now, let's see this in action!

### 6.2 Running the Edit Selected Text Tool

Let's see a practical example of using the `edit_selected_text` tool. We'll use the in-memory MCP client to call the tool directly.

First, let's import the MCP server and create an in-memory client:

In [19]:
from brown.mcp.server import mcp
from fastmcp import Client

# Create an in-memory client
mcp_client = Client(mcp)

2025-11-26 17:28:25.672 | INFO     | brown.mcp.server:<module>:34 - Initializing Brown MCP Server...
2025-11-26 17:28:25.674 | INFO     | brown.mcp.server:<module>:36 - Brown MCP Server initialized successfully


Now let's call the `edit_selected_text` tool on a specific set of paragraphs. We'll use the sample example when we ran the workflow directly as a LangGraph workflow.

Let's remember how it looks like:

In [20]:
article = MarkdownArticleLoader(uri="article.md").load(working_uri=SAMPLE_DIR)

start_line = 8
end_line = 42
selected_text = "\n".join(article.content.split("\n")[start_line:end_line])
pretty_print.wrapped(selected_text, title="Selected text to edit")

-------------------------------------- Selected text to edit --------------------------------------
  
To make the right choice, you first need to understand what LLM workflows and AI agents are. We will look at their core properties and how they are used, rather than their technical specifics.

### LLM Workflows

An LLM workflow is a sequence of tasks orchestrated by developer-written code. It can include LLM calls, but also other operations like reading from a database or calling an API. Think of it like a recipe where each step is explicitly defined. The key characteristic is that the path is determined in advance, resulting in a deterministic or rule-based system. This gives you predictable execution, explicit control over the application's flow, and makes the system easier to test and debug. Because you control every step, you know exactly where a failure occurred and how to fix it.

```mermaid
graph TD
    A["Start"] --> B["LLM Call"]
    B --> C["Process Data"]
    C --> D["Stor

Now let's call the tool with human feedback:

**Note how we get all these beautiful progress messages while we call the edit selected text workflow!**

In [21]:
async with mcp_client:
    result = await mcp_client.call_tool(
        "edit_selected_text",
        {
            "article_path": str(SAMPLE_DIR / "article.md"),
            "human_feedback": "Expand on the definition of both workflows and agents. Improve the first diagram.",
            "selected_text": selected_text,
            "first_line_number": start_line,
            "last_line_number": end_line,
        },
    )
    pretty_print.wrapped(
        result.data,
        title="Edit Selected Text Result",
    )

[11/26/25 17:28:25] DEBUG    Sending INFO to client: Editing selected text from file                 ]8;id=702713;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/server/context.py\context.py]8;;\:]8;id=792198;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/server/context.py#750\750]8;;\
                             inputs/tests/01_sample_small/article.md                                               

                    DEBUG    Sending INFO to client: Using directory `inputs/tests/01_sample_small`  ]8;id=762305;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/server/context.py\context.py]8;;\:]8;id=923375;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/server/context.py#750\750]8;;\
                             as context                                                                            

                    INFO     Received INFO from server: {'msg': 'Editing selected text from file      ]8;id=571742;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/client/logging.py\logging.py]8;;\:]8;id=703373;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/client/logging.py#44\44]8;;\
                             inputs/tests/01_sample_small/article.md', 'extra': None}                              

                    INFO     Received INFO from server: {'msg': 'Using directory                      ]8;id=780915;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/client/logging.py\logging.py]8;;\:]8;id=593038;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/client/logging.py#44\44]8;;\
                             `inputs/tests/01_sample_small` as context', 'extra': None}                            

OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=019ac0c7-b897-70a3-971f-16bb615f23a2&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


...

                    DEBUG    Sending INFO to client: 100%: Selected text editing completed           ]8;id=80669;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/server/context.py\context.py]8;;\:]8;id=921276;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/server/context.py#750\750]8;;\

                    INFO     Received INFO from server: {'msg': '100%: Selected text editing          ]8;id=432986;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/client/logging.py\logging.py]8;;\:]8;id=692862;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/client/logging.py#44\44]8;;\
                             completed', 'extra': None}                                                            

                    DEBUG    Sending INFO to client:                                                 ]8;id=868250;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/server/context.py\context.py]8;;\:]8;id=75235;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/server/context.py#750\750]8;;\
                             Here is the edited selected text:                                                     
                                                                                                                   
                             <selected_text>                                                                       
                                                                                                                   
                                 <content>```xml                                                                   
                             <selected_text>                                                                       
                                 <content>                                                                         
                             To make the right choice, you first need to understand what LLM                       
                             workflows and AI agents are. We will look at their core properties and                
                             how they are used, rather than their technical specifics.                             
                                                                                                                   
                             ### LLM Workflows                                                                     
                                                                                                                   
                             An LLM workflow is a sequence of tasks orchestrated by                                
                             developer-written code. It can involve LLM calls, reading from                        
                             databases, or calling APIs. Each step is explicitly defined, like a                   
                             detailed recipe.                                                                      
                                                                                                                   
                             This predefined path ensures deterministic, rule-based execution. You                 
                             get predictable results and explicit control, making workflows easier                 
                             to test and debug. You know exactly where failures occur and how to fix               
                             them.                                                                                 
                                                                                                                   
                             ```mermaid                                                                            
                             graph TD                                                                              
                                 A["Start"] --> B["Validate Input"]                                                
                                 B --> C["LLM Call"]                                                               
                                 C --> D["Process Data"]                                                           
                                 D --> E["Call External API"]                                                      
                                 E --> F["Store Data"]                                                             
                                 F --> G["End"]                                                  

[11/26/25 17:29:17] INFO     Received INFO from server: {'msg': '\nHere is the edited selected        ]8;id=623404;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/client/logging.py\logging.py]8;;\:]8;id=792673;file:///Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.venv/lib/python3.12/site-packages/fastmcp/client/logging.py#44\44]8;;\
                             text:\n\n<selected_text>\n    \n    <content>```xml\n<selected_text>\n                
                             <content>\nTo make the right choice, you first need to understand what                
                             LLM workflows and AI agents are. We will look at their core properties                
                             and how they are used, rather than their technical specifics.\n\n### LLM              
                             Workflows\n\nAn LLM workflow is a sequence of tasks orchestrated by                   
                             developer-written code. It can involve LLM calls, reading from                        
                             databases, or calling APIs. Each step is explicitly defined, like a                   
                             detailed recipe.\n\nThis predefined path ensures deterministic,                       
                             rule-based execution. You get predictable results and explicit control,               
                             making workflows easier to test and debug. You know exactly where                     
                             failures occur and how to fix them.\n\n```mermaid\ngraph TD\n                         
                             A["Start"] --> B["Validate Input"]\n    B --> C["LLM Call"]\n    C -->                
                             D["Process Data"]\n    D --> E["Call External API"]\n    E --> F["Store               
                             Data"]\n    F --> G["End"]\n```\nImage 1: A flowchart illustrating a                  
                             deterministic LLM workflow with clear start and end points, including an              
                             LLM call and data operations.\n\n### AI Agents\n\nAI agents are systems               
                             where an LLM dynamically decides the sequence of steps, reasoning, and                
                             actions to achieve a goal. Their path is not predefined. The agent plans              
                             its actions based on the task and its environment.\n\nThis dynamic                    
                             process, often modeled on frameworks like ReAct (Reason, Act, Observe),               
                             allows agents to be adaptive. They handle new situations through                      
                             LLM-driven autonomy, selecting tools, executing actions, and correcting               
                             their course                                                                          
                             [[1]](https://www.youtube.com/watch?v=kQxr-uOxw2o&t=1s).\n\nNow that we               
                             have a clear understanding of what LLM workflows and AI agents are,                   
                             let\'s explore their core differences and when to choose each approach                
                             for your AI applications.\n\n## Choosing Your Path</content>\n                        
                             <first_line_number>8</first_line_number>\n                                            
                             <last_line_number>42</last_line_number>\n</selected_text>\n```</content>              
                             \n    <first_line_number>8</first_line_number>\n                                      
                             <last_line_number>42</last_line_number>\n</selected_text>\n\n\nHere 

------------------------------------ Edit Selected Text Result ------------------------------------
  
Here is the edited selected text:

<selected_text>
    
    <content>```xml
<selected_text>
    <content>
To make the right choice, you first need to understand what LLM workflows and AI agents are. We will look at their core properties and how they are used, rather than their technical specifics.

### LLM Workflows

An LLM workflow is a sequence of tasks orchestrated by developer-written code. It can involve LLM calls, reading from databases, or calling APIs. Each step is explicitly defined, like a detailed recipe.

This predefined path ensures deterministic, rule-based execution. You get predictable results and explicit control, making workflows easier to test and debug. You know exactly where failures occur and how to fix them.

```mermaid
graph TD
    A["Start"] --> B["Validate Input"]
    B --> C["LLM Call"]
    C --> D["Process Data"]
    D --> E["Call External API"]
    E --> F

⚠️ **Important Observation**

Now the question is how do we apply these changes to the original article?

Note the text after the `selected_text` XML block we have the following instructions:
```text
Here is what you have to do with edited selected text:
- print the edited selected text to the console for the user to see
- give a quick summary of the changes you made
- always apply the changes to the source file without waiting for the user to explicitly ask you to do so
- in case you haven't changed anything, just say that you haven't changed anything
```

This instructs the tool consumer, which is the MCP client, such as Cursor, what to do with the new version of the selected text. In this use case, using these instructions, we instruct the MCP Client to always apply these changes to the original article. Otherwise, the edited selected text will be showed within the tool output, but never applied as a patch to the original article.

When using this with tools such as Cursor, it will show you the `diff` experience where you have to manually accept the new changes, improving even more the whole human in the loop experience.

### 6.3 MCP Prompts

MCP prompts provide pre-configured templates that MCP clients can use to easily trigger a tool or a composition of tools (remember the example from Nova the deep research agent). Brown exposes three prompts corresponding to the three tools as an easy way to interface with its functionality:


```python
@mcp.prompt
def generate_article_prompt(dir_path: Path) -> str:
    """Retrieve a prompt that will trigger the article generation workflow."""
    return f"""
Using Brown hosted as an MCP server, generate an article using all the necessary resources from 
the following directory: `{dir_path}`. Don't check if any expected files are missing, just trigger 
the "generate_article" tool of the Brown MCP Server, which will take care of everything.
"""

@mcp.prompt
def edit_article_prompt(human_feedback: str = "") -> str:
    """Retrieve a prompt that will trigger the article editing workflow."""
    return f"""
Using Brown hosted as an MCP server, edit an entire article based on human feedback 
and other expected requirements. Don't check if any expected files are missing, 
just trigger the "edit_article" tool of the Brown MCP Server, which will take care 
of everything.

Human feedback:
<human_feedback>
{human_feedback}
</human_feedback>

If the <human_feedback> is empty, you will infer it from the previous messages. If there are no other messages 
to infer from, use an empty string. Don't ever fill it in with things such as "Please provide more details" or
fill it in with generic stuff.
"""

@mcp.prompt
def edit_selected_text_prompt(human_feedback: str = "") -> str:
    """Retrieve a prompt that will trigger the selected text editing workflow."""
    return f"""
Using Brown hosted as an MCP server, edit the selected text from the article based on human feedback and 
other expected requirements. Don't check if any expected files are missing, just trigger the "edit_selected_text" 
tool of the Brown MCP Server, which will take care of everything.

Human feedback:
<human_feedback>
{human_feedback}
</human_feedback>

If the <human_feedback> is empty, you will infer it from the previous messages. If there are no other messages 
to infer from, use an empty string. Don't ever fill it in with things such as "Please provide more details" or
fill it in with generic stuff.
"""
```
These prompts make it easy for users to trigger the MCP's server tool without reading any documentation.

Now, let's see how we can access the prompts, starting with the one for generating articles:

In [22]:
async with mcp_client:
    result = await mcp_client.get_prompt(
        "generate_article_prompt",
        {
            "dir_path": SAMPLE_DIR,
        },
    )
    for message in result.messages:
        pretty_print.wrapped(
            [
                f"Role: {message.role}",
                f"Content: {str(message.content.text)}",
            ],
            title=f"`{message.role}` Message",
        )

------------------------------------------ `user` Message ------------------------------------------
  Role: user
----------------------------------------------------------------------------------------------------
  Content: 
Using Brown hosted as an MCP server, generate an article using all the necessary resources from 
the following directory: `inputs/tests/01_sample_small`. Don't check if any expected files are missing, just trigger 
the "generate_article" tool of the Brown MCP Server, which will take care of everything.

----------------------------------------------------------------------------------------------------


Now, let's look how the one for editing a piece of selected text looks like:

In [23]:
async with mcp_client:
    result = await mcp_client.get_prompt(
        "edit_selected_text_prompt",
        {
            "human_feedback": human_feedback,
        },
    )
    for message in result.messages:
        pretty_print.wrapped(
            [
                f"Role: {message.role}",
                f"Content: {str(message.content.text)}",
            ],
            title=f"`{message.role}` Message",
        )

------------------------------------------ `user` Message ------------------------------------------
  Role: user
----------------------------------------------------------------------------------------------------
  Content: 
Using Brown hosted as an MCP server, edit the selected text from the article based on human feedback and 
other expected requirements. Don't check if any expected files are missing, just trigger the "edit_selected_text" 
tool of the Brown MCP Server, which will take care of everything.

Human feedback:
<human_feedback>
{"content":"Make the introduction more engaging and catchy. \nAlso, expand on the definition of both workflows and agents from the first section"}
</human_feedback>

If the <human_feedback> is empty, you will infer it from the previous messages. If there are no other messages 
to infer from, use an empty string. Don't ever fill it in with things such as "Please provide more details" or
fill it in with generic stuff.

-------------------------------

The `edit_article_prompt` is almost identical to `edit_selected_text_prompt`. Thus, we will skip it.

Still, the real beauty of using prompts through MCP Servers is when we actually plug in the server into a client such as Cursor that has access to a chatbot that can pick up on the instructions from the retrieved prompts and follows the instructions from it such as calling the generate article tool.

This makes prompts an amazing way to interface with MCP Server, for example describing workflows that use the MCP Server tools in various ways, and passing them directly to the client, without letting the consumer worry about how to use them.

We will illustrate this in the video attached to this lesson where we will show how this works in Cursor, but a similar behavior applies in other MCP clients, such as Claude.

### 6.4 MCP Resources

MCP resources provide read-only access to configuration and state information. Brown exposes resources for accessing the app configuration and profiles:
```python
@mcp.resource("resource://config/app", mime_type="application/json")
def get_app_config() -> dict:
    """Get the application configuration for Brown Agent as an MCP resource.
    
    Returns:
        dict: The application configuration including:
            - Model configurations for each workflow node
            - File paths for profiles, examples, and context files
            - Number of review iterations and workflow settings
            - Temperature and other model parameters
    """
    return app_config.model_dump(mode="json")

@mcp.resource("resource://profiles/character")
def get_character_profile() -> str:
    """Get the character profile resource for Brown Agent.
    
    Returns:
        str: The character profile content in markdown format
    """
    return __get_profile("character")
```

Resources allow MCP clients to understand how Brown is configured without modifying any state.

Let's call the resources to see the MCP server configuration:

In [24]:
async with mcp_client:
    resources = await mcp_client.list_resources()

    for resource in resources:
        resource_dict = {
            "uri": str(resource.uri),
            "name": resource.name,
            "description": resource.description,
            "mime_type": resource.mimeType,
        }
        if hasattr(resource, "_meta") and resource._meta:
            fastmcp_meta = resource._meta.get("_fastmcp", {})
            resource_dict["tags"] = fastmcp_meta.get("tags", [])

        pretty_print.wrapped(
            resource_dict,
            title=f"`{resource.name}` Resource",
        )

------------------------------------ `get_app_config` Resource ------------------------------------
  {
  "uri": "resource://config/app",
  "name": "get_app_config",
  "description": "Get the application configuration for Brown Agent as an MCP resource.\n\nThis resource provides access to the complete Brown Agent configuration,\nincluding model settings, file paths, and workflow parameters. The configuration\nis loaded from YAML files and converted to a JSON-serializable format.\n\nReturns:\n    dict: The application configuration as a JSON-serializable dictionary containing:\n        - Model configurations for each workflow node (write_article, review_article, etc.)\n        - File paths for profiles, examples, and context files\n        - Number of review iterations and workflow settings\n        - Temperature and other model parameters\n        - Tool configurations and model assignments",
  "mime_type": "application/json"
}
----------------------------------------------------------

Now, let's look into each resource independently starting with the app configuration:

In [25]:
import json

async with mcp_client:
    content = await mcp_client.read_resource("resource://config/app")

    pretty_print.wrapped(
        json.loads(content[0].text),
        title="App Configuration",
    )

---------------------------------------- App Configuration ----------------------------------------
  {
  "context": {
    "article_guideline_loader": "markdown",
    "article_guideline_uri": "article_guideline.md",
    "research_loader": "markdown",
    "research_uri": "research.md",
    "article_loader": "markdown",
    "article_renderer": "markdown",
    "article_uri": "article.md",
    "profiles_loader": "markdown",
    "profiles_uri": "inputs/profiles",
    "character_profile": "paul_iusztin.md",
    "examples_loader": "markdown",
    "examples_uri": "inputs/examples/course_lessons"
  },
  "memory": {
    "checkpointer": "in_memory"
  },
  "num_reviews": 2,
  "nodes": {
    "generate_media_items": {
      "model_id": "google_genai:gemini-2.5-flash",
      "config": {
        "temperature": 0.0,
        "top_k": null,
        "n": 1,
        "response_modalities": null,
        "include_thoughts": false,
        "thinking_budget": null,
        "max_output_tokens": null,
        "m

Now, the character profile:

In [26]:
async with mcp_client:
    content = await mcp_client.read_resource("resource://profiles/character")

    pretty_print.wrapped(
        content[0].text,
        title="Character Profile",
    )

---------------------------------------- Character Profile ----------------------------------------
  ## About Paul Iusztin

I’m **Paul Iusztin**. A 29 years old senior AI Engineer and content creator.

I help engineers ship AI products.

I’m the author of the bestseller LLM Engineer’s Handbook, lead instructor of the Agentic AI Engineering course, founding AI Engineer of a San Francisco start-up, and obsessed with making knowledge accessible through AI.

With over 10 years of experience and 20 apps shipped, I teach AI Engineering as I wanted to at the beginning of my career. End-to-end. From idea to production. From data collection to deploying, monitoring and evaluation. With a focus on AI principles, software patterns and infrastructure systems that will thrive in a future dominated by AI coding tools.

My ultimate goal is to help other engineers escape the PoC purgatory and x10 their AI Engineering skills.

I'm also the founder of the **Decoding AI Magazine**, a place for real-worl

### 6.5 Clean Separation of MCP Layer

Notice how all the MCP code lives in the `brown.mcp` module and simply imports from other layers:

- **Domain layer**: Imports entities like `Article`, `HumanFeedback`, `Review`
- **App layer**: Imports workflows like `build_edit_article_workflow`, `build_edit_selected_text_workflow`
- **Infrastructure layer**: Imports builders like `build_loaders`, `build_short_term_memory`

The MCP layer is just a thin serving/interface layer that orchestrates the other components. This separation means:

- We can serve Brown through different methods (CLI, FastAPI, gRPC) without changing core logic
- The MCP code is independent and can be modified without affecting other layers
- We can easily add new MCP tools, prompts, or resources
- Testing is easier because each layer has clear boundaries

This is a key advantage of using clean architecture principles throughout the Brown project.

## 7. Using Brown's MCP Server Through the CLI

Brown includes a command-line interface that lets you interact with the MCP server directly from your terminal. This is useful for scripting, automation, and quick testing.

The CLI script is located at `lessons/writing_workflow/scripts/brown_mcp_cli.py` and provides three main commands:

1. **generate-article**: Generate an article from scratch
2. **edit-article**: Edit an entire article based on human feedback
3. **edit-selected-text**: Edit a selected section of an article

⚠️ You can run these only from the Brown standalone project from `lessons/writing_workflow`.

### Usage Examples

Here is how the CLI commands look like:
```bash
# Generate an article
python scripts/brown_mcp_cli.py generate-article --dir-path /path/to/article

# Edit an entire article
python scripts/brown_mcp_cli.py edit-article \
    --dir-path /path/to/article \
    --human-feedback "Make the introduction more engaging"

# Edit selected text
python scripts/brown_mcp_cli.py edit-selected-text \
    --dir-path /path/to/article \
    --human-feedback "Make this shorter. Remove all em-dashes." \
    --first-line 10 \
    --last-line 20
```

The CLI uses the in-memory MCP client to call the MCP server tools, providing a simple interface for Brown's capabilities.

### How to run it?

> [!NOTE]
> 💡 For more details on running these CLI commands while using Brown as a standalone project, see the documentation from [lessons/writing_workflow](../writing_workflow/).

## 8. Using Brown's MCP Server Through the Cursor IDE

The real power of Brown comes when integrating it with MCP clients like Cursor. This creates a writing experience similar to coding with AI assistance.

### 8.1 Cursor Configuration

To use Brown with Cursor, you need to configure it in your `.cursor/mcp.json` file (similar to what we did for Nova, the research agent).

Open up this repository with Cursor and add the following config within your `.cursor/mcp.json` file. Create it, if it doesn't exists:

```json
{
    "mcpServers": {
        "brown": {
            "command": "uv",
            "args": [
                "--directory",
                "lessons/writing_workflow/",
                "run",
                "python",
                "-m",
                "brown.mcp.server"
            ],
            "cwd": "${workspaceFolder}",
            "env": {
                "ENV_FILE_PATH": "${workspaceFolder}/.env"
            }
        }
    }
}
```

...or if you already have other `mcpServers` added like the deep research agent, add `brown` along them, like this:

```json
{
    "mcpServers": {
        "nova": {
            "command": "uv",
            "args": [
                "--directory",
                "lessons/research_agent_part_2/mcp_server",
                "run",
                "-m",
                "src.server",
                "--transport",
                "stdio"
            ],
            "cwd": "${workspaceFolder}",
            "env": {
                "ENV_FILE_PATH": "${workspaceFolder}/.env"
            }
        },
        "brown": {
            "command": "uv",
            "args": [
                "--directory",
                "lessons/writing_workflow/",
                "run",
                "python",
                "-m",
                "brown.mcp.server"
            ],
            "cwd": "${workspaceFolder}",
            "env": {
                "ENV_FILE_PATH": "${workspaceFolder}/.env"
            }
        }
    }
}

```

This configuration tells Cursor:

- The name of the MCP server ("brown", "nova")
- How to launch the server using `uv` and Python
- The working directory
- Where to load the environment variables. Ensure your `.env` file is located at `${workspaceFolder}/.env`, where `${workspaceFolder}` is the root of the project.

Once configured, go to Cursor's settings `Tools & MCP` section and ensure it has discovered `brown` (with a green light) as an MCP server with tools, prompts and resources available.

More on how to use Brown with Cursor in the video from the lesson!


### 8.2 The Brown + Human-in-the-Loop Writing Experience

With Brown integrated into Cursor, your writing workflow becomes:

1. **Generate Initial Article**: Use the `generate_article` tool or prompt to create a first draft
2. **Review as Human Expert**: Read through the generated article with your domain expertise
3. **Provide Feedback**: Select sections that need improvement and provide specific feedback
4. **AI Edits**: Use `edit_article` or `edit_selected_text` tools to refine based on your feedback
5. **Iterate**: Repeat steps 2-4 until satisfied with the article

This creates a collaborative experience where:
- AI handles the heavy lifting of writing and editing
- You guide the direction with your expertise and feedback
- The workflow is fast and iterative
- You maintain full control over the final output

## 9. Running Brown Through the MCP Server (Video)

We've created a video demonstration showing how to use Brown's human-in-the-loop features, while using it through the MCP Server in Cursor.

[Link to video demonstration will be provided]

## 10. Conclusion

In this lesson, we've explored how to implement human-in-the-loop capabilities in the Brown writing workflow. Let's recap what we've learned and look at future possibilities.

### What We've Learned

1. **Human-in-the-Loop**: We learned how to design AI workflows that balance automation with human expertise, allowing AI to handle heavy lifting while keeping humans in control

2. **Edit Article Workflow**: We implemented the `edit_article` workflow that reviews and edits entire articles based on human feedback and expected requirements

3. **Edit Selected Text Workflow**: We built the `edit_selected_text` workflow for precise, focused edits on specific article sections

4. **MCP Server**: We served Brown as an MCP server with tools, prompts, and resources while respecting clean architecture principles

5. **Cursor Integration**: We learned how to integrate Brown with Cursor to create a coding-like writing experience with human-in-the-loop editing

The key insight is that we can use a low number of review loops during initial article generation, then dynamically run additional editing workflows with human feedback when necessary. This approach is both efficient and effective.

### Future Extension Ideas

Here are some ways you can extend Brown to fit your needs:

1. **Hook Brown to Claude Desktop**: Instead of using Cursor, integrate Brown with Claude Desktop for a different AI assistant experience

2. **Use Resource Templates** to parameterize the writing profiles and easily add support for all the available profiles. [More here](https://gofastmcp.com/servers/resources#resource-templates).

3. **Serve Brown through FastAPI**: Replace the MCP server with a FastAPI REST API for web-based integrations

4. **Add Guideline Review Tool**: Create a workflow to review and edit your article guidelines themselves, helping you refine your inputs

### Wrapping Up Brown

With this lesson, we've wrapped up our exploration of the Brown writing workflow. We've covered:

- **Lesson 22**: Foundation of the writing workflow with entities, nodes, and basic workflows
- **Lesson 23**: The evaluator-optimizer pattern with article reviews and iterative refinement
- **Lesson 24**: Human-in-the-loop implementation with MCP server integration

Together, these lessons show how to build a production-ready AI writing system that:
- Generates high-quality articles from research and guidelines
- Uses multiple review loops to ensure quality
- Integrates human feedback for iterative refinement
- Exposes capabilities through standard protocols like MCP
- Maintains clean architecture for extensibility and maintainability

In future lessons, we'll explore how to orchestrate Nova and Brown as a multi-agent system. 

### Resources

- **Brown Package**: Explore `lessons/writing_workflow`
- **Configuration Examples**: Check `configs/` for different configurations
- **Test Data**: Use `inputs/tests/` for additional testing scenarios
- **Writing Profiles**: Check `inputs/profiles/` for more profile templates